# Example of simple fit with PyTorch

* First implement in Numpy and with scipy.optimize
* Then try to do the same with PyTorch and TensorFlow
* Then try to do something more interesting with gamma-ray data

In [22]:
import numpy as np
from scipy.optimize import minimize

## Example data

Best-fit line has pars = (2, 0), i.e. slope 2, and intercept 0

In [23]:
x = np.array([1, 2, 3])
y = np.array([2, 4, 6])
data = {'x': x, 'y': y}

## Numpy & Scipy

Define cost function using Numpy; fit with scipy.optimize.minimize

In [25]:
def cost(pars, data):
    y_model = pars[0] * data['x'] + pars[1]
    dy = data['y'] - y_model
    return np.sum(dy ** 2)

pars = (42, 42) # any incorrect start values
res = minimize(cost, pars, args=(data,))
print(res) # correct result: slope = 2, intercept: 0

      fun: 5.602873721148142e-13
 hess_inv: array([[ 0.24995886, -0.50004917],
       [-0.50004917,  1.16660792]])
      jac: array([-5.37158973e-06, -2.43206103e-06])
  message: 'Optimization terminated successfully.'
     nfev: 32
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([ 1.99999984e+00, -9.94556075e-08])


## PyTorch

Let's try to do the same thing using PyTorch

In [26]:
import torch

## Tensorflow

Let's try to do the same thing using TensorFlow

In [27]:
import tensorflow as tf